In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Leitura e Tratamento dos Dados

In [ ]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
display(train)
display(test.head())

In [ ]:
print(f'Train set shape: {train.shape}\nTest set shape: {test.shape}')

In [ ]:
train['Embarked'].value_counts()

In [ ]:
def transforma_embarked(feature):
    if feature == 'S':
        return 0
    elif feature == 'C':
        return 1
    else:
        return 2


train['Embarked_b'] = train['Embarked'].map(transforma_embarked)

In [ ]:
train['Embarked_b'].value_counts()

In [ ]:
variaveis = ['Sex_b', 'Age', 'Pclass', 'Ticket']

In [ ]:
train['Sex'].value_counts()

In [ ]:
def transforma_sexo(feature):
    if feature == 'female':
        return 1
    else:
        return 0


train['Sex_b'] = train['Sex'].map(transforma_sexo)

In [ ]:
display(train.head())
train['Sex_b'].value_counts()

In [ ]:
train['Cabin'].value_counts()

In [ ]:
train.info()

In [ ]:
X_train = train[['Sex_b', 'Age', 'Pclass', 'Embarked_b', 'SibSp', 'Parch', 'Fare']]
y_train = train['Survived']

In [ ]:
X_train = X_train.fillna(-1)
X_train

In [ ]:
y_train

In [ ]:
X_train_s, X_cv, y_train_s, y_cv = train_test_split(X_train, y_train, test_size=0.4,
                                                      random_state=42)

In [ ]:
X_train_s.shape, X_cv.shape, y_train_s.shape, y_cv.shape

In [ ]:
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train_s)
X_train_norm, X_train_s

In [ ]:
X_cv_norm = scaler.transform(X_cv)
X_cv_norm, X_cv

In [ ]:
train['Name']

# Regressão Logística

In [ ]:
model = LogisticRegression()
model.fit(X_train_norm, y_train_s)
predict = model.predict(X_train_norm)

In [ ]:
acc = model.score(X_train_norm, y_train_s)
f'{acc:.2f}'

In [ ]:
predict_cv = model.predict(X_cv_norm)
acc_cv = model.score(X_cv_norm, y_cv)
acc_cv

In [ ]:
test['Sex_b'] = test['Sex'].map(transforma_sexo)
test['Embarked_b'] = test['Embarked'].map(transforma_embarked)

In [ ]:
X_test = test[['Sex_b', 'Age', 'Pclass', 'Embarked_b', 'SibSp', 'Parch', 'Fare']].fillna(-1)

In [ ]:
X_test_norm = scaler.transform(X_test)

In [ ]:
predict_t = model.predict(X_test_norm)

# Criando o Resultado das Previsões para Importar ao Kaggle

In [ ]:
result_t = pd.Series(predict_t, index=test['PassengerId'], name='Survived')
result_t

In [ ]:
result_t.to_csv('./yhat/logistic_regression.csv', header=True)